<a href="https://colab.research.google.com/github/WeixuanZ/shipping-classifier/blob/master/shipping_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [0]:
!test -d bert || git clone https://github.com/google-research/bert bert
!test -d shipping-classifier || git clone https://github.com/WeixuanZ/shipping-classifier.git

!cd bert && test -d dataset || mkdir dataset
!cd bert && test -d model || mkdir model
!cd bert && test -d bert_output || mkdir bert_output
!cd bert/model && test -f bert_config.json || (wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip && unzip *.zip && mv uncased_L-12_H-768_A-12/* . && rm -rf uncased_L-12_H-768_A-12.zip uncased_L-12_H-768_A-12)
# !cd bert/model && rm -f * && test -f bert_config.json || (wget https://storage.googleapis.com/bert_models/2019_05_30/wwm_uncased_L-24_H-1024_A-16.zip && unzip *.zip && mv wwm_uncased_L-24_H-1024_A-16/* . && rm -rf wwm_uncased_L-24_H-1024_A-16.zip wwm_uncased_L-24_H-1024_A-16)


In [0]:
!cd bert && rm run_classifier.py && cp ../shipping-classifier/bert/run_classifier.py .
!cd bert/dataset && cp -r ../../shipping-classifier/bert/dataset/* .

In [0]:
!rm -rf ./bert/bert_output/ && mkdir ./bert/bert_output/

In [0]:
!cd ./bert && python3 run_classifier.py --task_name=cola --do_train=true --do_eval=true --data_dir=./dataset --vocab_file=./model/vocab.txt --bert_config_file=./model/bert_config.json --init_checkpoint=./model/bert_model.ckpt --max_seq_length=64 --train_batch_size=4 --learning_rate=3e-5 --num_train_epochs=3.0 --output_dir=./bert_output/ --do_lower_case=true --save_checkpoints_steps 2000

In [0]:
import argparse
import os
import uuid

import pandas as pd

# text = ['plague inc', 'arm', 'autodesk ltd', 'apple', 'samsung', 'tianjin']
text = []

def test():
    !cd ./bert && python3 run_classifier.py --task_name=cola --do_predict=true --data_dir=./dataset --vocab_file=./model/vocab.txt --bert_config_file=./model/bert_config.json --init_checkpoint=./bert_output/model.ckpt-1525 --max_seq_length=64 --output_dir=./bert_output/

    df_test = pd.read_csv('./bert/dataset/test.tsv', sep='\t')
    df_test_with_label = pd.read_csv('./bert/dataset/test_with_label.tsv', sep='\t')

    df_result = pd.read_csv('./bert/bert_output/test_results.tsv', sep='\t', header=None)

    df_map_result = pd.DataFrame({
        'text': df_test['text'],
        'label': df_test_with_label['label'],
        'prediction': [['port', 'date', 'vessel', 'company'][i] for i in df_result.idxmax(axis=1)]
    })

    print(df_map_result)


if len(text) == 0:
    test()
else:
    try:
        os.makedirs('./bert/cache')
    except FileExistsError:
        pass
    identifier = []
    for i in text:
        identifier.append(uuid.uuid4())
    df = pd.DataFrame({'guid': identifier, 'text': text})
    df.to_csv('./bert/cache/test.tsv', sep='\t', index=False, header=True)

    !cd ./bert && python3 run_classifier.py --task_name=cola --do_predict=true --data_dir=./cache --vocab_file=./model/vocab.txt --bert_config_file=./model/bert_config.json --init_checkpoint=./bert_output/model.ckpt-1525 --max_seq_length=64 --output_dir=./cache/
    df_result = pd.read_csv('./bert/cache/test_results.tsv', sep='\t', header=None)
    df_map_result = pd.DataFrame({
        'text': df['text'],
        'prediction': [['port', 'date', 'vessel', 'company'][i] for i in df_result.idxmax(axis=1)]
    })
    print(df_map_result)